In [ ]:
#Import the necessary libraries

import ee
import math
import datetime
import os


ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=B7KhUyKXRrxaifhW0amXCdx6NwkSwQ03z9_cMC4CDEU&tc=HXC_sxqERrh7EI1s3_9a2csv7KncA2I6M4LGz42QPlE&cc=50C0y4hPjoiRFT2Br0wtXKSPvixC4cWyfnwpDWWDKmk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qiIxiFuBrN_QzZBQmZFFHSx-vVLQSvNDmkaWHgMSfcOBBXu4PWxqPw

Successfully saved authorization token.


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!wget https://raw.githubusercontent.com/adugnag/S1-S2_Transformer/main/helper.py

from helper import s1_preproc, get_s2_sr_cld_col, apply_cld_shdw_mask, add_cld_shdw_mask, NBRaddTimeline

In [ ]:
#parameters
params = {  'START_DATE': '2019-01-01', 
            'STOP_DATE': '2020-12-31',        
            'ORBIT': 'DESCENDING',
            'RELATIVE_ORBIT_NUMBER': 'ANY', 
            'SATELLITE': 'ANY',
            'POLARIZATION': 'VVVH',
            'ROI': ee.Geometry.Polygon([[[-63.166, -17.792],[-63.166, -19.157],[-61.535, -19.157],[-61.535, -17.792]]]),
            'DEM': ee.Image('USGS/SRTMGL1_003'),
            'APPLY_BORDER_NOISE_CORRECTION': True,
            'APPLY_TERRAIN_FLATTENING': False,
            'TERRAIN_FLATTENING_MODEL': 'VOLUME',
            'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
            'APPLY_SPECKLE_FILTERING': True,
            'SPECKLE_FILTER_FRAMEWORK':'MULTI',
            'SPECKLE_FILTER': 'BOXCAR',
            'SPECKLE_FILTER_KERNEL_SIZE': 15,
            'NR_OF_IMAGES':10,
            'FORMAT': 'DB',
            'CLIP_TO_ROI': False,
          'interval':30,
          'increment':'day',
          'OUTPUT_BUCKET' : 'senalerts_dl4',
          'FOLDER':'TDF1',
          'BANDS' : ['VV','VH','NBR'],
          'LABEL_DATA' : ee.FeatureCollection("projects/ee-adugnagirma/assets/Samples_SA2_binary_bg40p_nf10p_Fhansen_5000_SAVED_COLL_0708_desc"),
          'LABEL' : ['Label'],
          'TRAIN_FILE_PREFIX' : 'Training_TDF_BOL_S1_desc_S2_v3',
          'TEST_FILE_PREFIX' : 'Testing_TDF_BOL_S1_desc_S2_v3',
          'VALIDATION_FILE_PREFIX' :'Validation_TDF_BOL_S1_desc_S2_v3'
            }


Number of images in the collection:  249


In [ ]:
#Features
FEATURE_NAMES = params['BANDS']
FEATURE_NAMES.append(params['LABEL'])

#Data path
TRAIN_FILE_PATH = 'gs://' + params['OUTPUT_BUCKET'] + '/' + params['FOLDER'] + '/' + params['TRAIN_FILE_PREFIX'] + '.tfrecord.gz'
TEST_FILE_PATH = 'gs://' + params['OUTPUT_BUCKET'] + '/'+ params['FOLDER'] + '/'  + params['TEST_FILE_PREFIX'] + '.tfrecord.gz'
VALIDATION_FILE_PATH = 'gs://' + params['OUTPUT_BUCKET'] + '/'+ params['FOLDER'] + '/'  + params['VALIDATION_FILE_PREFIX'] + '.tfrecord.gz'


In [ ]:
#process Sentinel 1 image collection to ARD level
s1_processed = s1_preproc(params)
s1_processed = s1_processed.select(['VV','VH'])
s1_nr = s1_processed.size().getInfo()

print('Number of images in the collection: ', s1_processed.size().getInfo())

#apply a moving median smoothing over the time-series
startDate = ee.Date(params['START_DATE']);
secondDate = startDate.advance(params['interval'], params['increment']).millis();
increase = secondDate.subtract(startDate.millis());
LIST1 = ee.List.sequence(startDate.millis(), ee.Date('2020-12-31').millis(), increase);

def mov_s1(date):
  return s1_processed.filterDate(ee.Date(date), ee.Date(date).advance(params['interval'], params['increment'])) \
           .median().set('system:time_start',ee.Date(date).millis())

#smoothened collection
collection1 =  ee.ImageCollection.fromImages(LIST1.map(mov_s1));

#convert the collection to a single image
image1 = collection1.toArrayPerBand()

#Select image to display
image_S1 = collection1.first()
print('Number of images in the smoothened S1 collection: ', collection1.size().getInfo())
 

Number of images in the smoothened S1 collection:  25


In [ ]:
#Sentinel-2 collection ARD processing

s2_sr_cld_col = get_s2_sr_cld_col(params['ROI'], params['START_DATE'], params['STOP_DATE'])
s2_sr = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
s2_sr = s2_sr.map(NBRaddTimeline).select('NBR')

print('Number of images in the original collection: ', s2_sr.size().getInfo())

#S2 time-series smoothing
def mov_s2(date):
  return s2_sr.filterDate(ee.Date(date), ee.Date(date).advance(params['interval'], params['increment'])) \
           .median().set('system:time_start',ee.Date(date).millis())

#apply time series smoothing
collection2 =  ee.ImageCollection.fromImages(LIST1.map(mov_s2));

image2 = collection2.toArrayPerBand()
s2_nr = s2_sr.size().getInfo()
print('Number of images in the smoothened S2 collection: ', collection2.size().getInfo())

Number of images in the original collection:  1413
Number of images in the smoothened S2 collection:  25


In [ ]:
#combine the sentinel-1 and 2 images for export
image = ee.Image.cat([image1,image2])

In [ ]:
#Optional visualizing 
import folium

visparam = {'bands':'VH', 'min': -25,'max': -5}

s1_processed_mapid = image1.getMapId(visparam)
label_mapid = params['LABEL_DATA'].getMapId()

map = folium.Map(location=[-21.22, -60.31])

folium.TileLayer(
    tiles=s1_processed_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Preprocessed S1',
  ).add_to(map)
folium.TileLayer(
  tiles=label_mapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='samples',
).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:

#random sampling from images
sample = image.sampleRegions(collection=params['LABEL_DATA'], properties=params['LABEL'], scale=10, tileScale=16).randomColumn()

#split training, validation and testing data
training = sample.filter(ee.Filter.lt('random', 0.8))
validation = sample.filter(ee.Filter.gt("random",0.9))
testing = sample.filter(ee.Filter.gt("random",0.8)).filter(ee.Filter.lt("random",0.9))

In [ ]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix= params['FOLDER'] + '/' + params['TRAIN_FILE_PREFIX'],
  bucket=params['OUTPUT_BUCKET'],
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix= params['FOLDER'] + '/' + params['TEST_FILE_PREFIX'],
  bucket=params['OUTPUT_BUCKET'],
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

validation_task = ee.batch.Export.table.toCloudStorage(
  collection=validation,
  description='Validation Export',
  fileNamePrefix= params['FOLDER'] + '/' + params['VALIDATION_FILE_PREFIX'],
  bucket=params['OUTPUT_BUCKET'],
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [ ]:
# Start the tasks.
training_task.start()
testing_task.start()
validation_task.start()